<a href="https://colab.research.google.com/github/LavineLin/Soft-Coral-Environmental-Acclimation-Insights-from-Microbiome-and-Metabolome-Dynamics/blob/main/%E3%80%8CMicrobiome_data_analysis_20240707%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tool

In [ ]:
!git clone https://github.com/schonkopf/soundscape_IR.git
!pip install umap-learn
!pip install -U kaleido
!pip install -U csaps

Cloning into 'soundscape_IR'...
remote: Enumerating objects: 2511, done.
remote: Counting objects: 100% (611/611), done.
remote: Compressing objects: 100% (332/332), done.
remote: Total 2511 (delta 391), reused 466 (delta 279), pack-reused 1900 (from 1)
Receiving objects: 100% (2511/2511), 74.05 MiB | 15.01 MiB/s, done.
Resolving deltas: 100% (1651/1651), done.
Updating files: 100% (56/56), done.


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
import umap.umap_ as umap
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from csaps import csaps
from sklearn.decomposition import non_negative_factorization as NMF
import matplotlib.pyplot as plt
from IPython.display import Image
import warnings
warnings.filterwarnings("ignore")

def nmf_rmse(input_data, max_n_basis, random_state=40):
  rmse=np.zeros([1,max_n_basis])[0]
  for n in np.arange(1,max_n_basis+1):
    W, H, _ = NMF(input_data, n_components=n, init='random', random_state=40, update_H=True, alpha_W=0, beta_loss='frobenius', solver='cd', max_iter=200)
    reconstruction = np.dot(W, H)
    rmse[n-1]=np.power(np.mean(np.power(input_data-reconstruction,2)),0.5)

  #rmse=-1*np.diff(rmse)
  ax.plot(np.arange(1,max_n_basis+1),rmse)
  _ = ax.set_xlabel('Number of basis')
  _ = ax.set_ylabel('RMSE')
  return rmse


# Load data
Exclude OTUs with total abundance (all samples) <1000

In [ ]:
path='.'

abundance_threshold=100
taxa_sort='OTU' # Set to 'OTU'|'Species'|'Genus'|'Family'

In [ ]:
data=pd.read_csv(path+'/'+'feature.csv', header=0)
abundance=np.sum(data.iloc[:,1:], axis=1)
process_list=np.where(abundance>=abundance_threshold)[0]
data=data.iloc[process_list,:]
data

In [ ]:
metadata=pd.read_csv(path+'/'+'metadata.csv', header=0)
metadata

In [ ]:
taxa=pd.read_csv(path+'/'+'remove_empty_raw_data_env_tax.csv', header=0)
taxa=taxa.iloc[process_list,:]
taxa

In [ ]:
if taxa_sort=='OTU':
  print('No sorting')
else:
  taxa_data=pd.concat([taxa, data], axis=1)
  taxa_data=taxa_data.groupby(taxa_sort).sum().reset_index()
  data=taxa_data.iloc[:,len(taxa.columns):]
  data['#OTU']=taxa_data[taxa_sort]
  taxa=taxa_data[taxa_sort]
data

# Identify OTU groups
Check OTU patterns across samples and treatments






## Use UMAP to analyze OTU heterogeneity
* **log_transform:** log10(n+1)
* **scale_normalize:** normalize the OTU abundance data of each sample to 0 and 1
* **UMAP_min_distance:** a value close to 0 will focus on regional variation (tend to overfit), and a value close to 1 will focus on global variation (avoid overfitting)


In [ ]:
log_transform=True
scale_normalize=False # not very effective for True
UMAP_min_distance=0.9 # between 0 and 1

# Data transformation
if log_transform:
    scaled_data=np.log10(data.iloc[:,1:]+1)
else:
    scaled_data=data.iloc[:,1:]

# Normalize each sample to 0-1
if scale_normalize:
    scaled_data=(scaled_data-scaled_data.min(axis=0))
    scaled_data=scaled_data/scaled_data.max(axis=0)

# Run UMAP analysis
umap_otu=umap.UMAP(n_components=1, metric='correlation', min_dist=UMAP_min_distance, random_state=10).fit(scaled_data)
otu_vector=np.argsort(umap_otu.embedding_[:,0])

# Save UMAP coordinates
umap_result=pd.DataFrame(umap_otu.embedding_)
umap_result=pd.concat([umap_result, taxa.reset_index(drop=True)],axis=1)
umap_result.to_csv('UMAP_OTU.csv', sep=',')

## Visualize UMAP result
Produce a heatmap of mean abundance (bin determined by *umap_bin_width*) to summarize OTU patterns across samples and treatments

In [ ]:
umap_bin_width=1 # value >0

# Create feature matrix
k=0
pick_umap=np.arange(np.floor(umap_result[0].min()),np.ceil(umap_result[0].max()),umap_bin_width)
W=np.zeros((len(pick_umap),scaled_data.shape[1]))+np.nan
for n in pick_umap:
    ind=np.where((umap_result[0]>=n)*(umap_result[0]<(n+umap_bin_width))==1)[0]
    W[k,:]=np.mean(scaled_data.iloc[ind,:],axis=0)
    k+=1

# Interactive visualization
fig = make_subplots(rows=1, cols=1)
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'white'})
fig.add_trace(go.Heatmap(z=W, y=pick_umap+umap_bin_width/2, x=list(data.columns[1:]), coloraxis='coloraxis1'), row=1, col=1)
fig.update_yaxes(title_text='UMAP Coordinate', range=[np.floor(umap_result[0].min()), np.ceil(umap_result[0].max())], showgrid=False, zeroline=False, showline=True, linewidth=2, linecolor='black', mirror=True, row=1, col=1)
fig.update_xaxes(title_text='Sample', showgrid=False, zeroline=False, showline=True, linewidth=2, linecolor='black', mirror=True, row=1, col=1)
fig.update_layout(coloraxis1=dict(colorscale='Jet', showscale=True), bargap=0.05)
fig.show()
fig.write_html(file='UMAP_OTU_1D.html')

# Active the foll
save_figure=True
if save_figure:
  fig.write_image("UMAP_OTU_1D.jpg", height=400, width=1500, scale=5)

## Use Non-negative matrix factorization to group OTUs into clusters

Run a series of NMF to find a minimum number of clusters that can maximize the reconstruction performance

In [ ]:
fig, ax=plt.subplots()
rmse=nmf_rmse(input_data=scaled_data, max_n_basis=15, random_state=10)

## Visualize the activation scores of each OTU cluster

In [ ]:
OTU_clusters=4 # int >1

W, H, _ = NMF(scaled_data, n_components=OTU_clusters, init='random', random_state=10, update_H=True, alpha_W=0, beta_loss='frobenius', solver='cd', max_iter=200)
W_cluster=np.argmax(W, axis=1)

fig = go.Figure(data=go.Heatmap(z=H, x=list(data.columns[1:]), y=np.arange(H.shape[0]), colorscale='Jet'))
fig.show()
fig.write_html(file='NMF_OTU_1D.html')

save_figure=True
if save_figure:
  fig.write_image("NMF_OTU_1D.jpg", height=400, width=1500, scale=5)

## Save UMAP and NMF results

In [ ]:
umap_result.insert(1, 'Cluster', W_cluster)
umap_result.to_csv('UMAP_OTU.csv', sep=',')
umap_result

# Explore the temporal heterogeneity of treatments

## Use UMAP to analyze sample heterogeneity

*   **select_clusters:** Select OTU clusters that are highly related to treatmeants
*   **UMAP_n_neighbors:** like knn clustering
*   **UMAP_min_distance:** a value close to 0 will focus on regional variation (tend to overfit), and a value close to 1 will focus on global variation (avoid overfitting)




In [ ]:
select_clusters=[0,2,3] # a list of int
UMAP_n_neighbors=15 # int >1
UMAP_min_distance=0.9 # between 0 and 1

otu_cluster=pd.read_csv('UMAP_OTU.csv', header=0)
otu_select=np.where(otu_cluster['Cluster'].isin(select_clusters))[0]
#otu_select=np.arange(len(otu_vector)) # Active this line if you want to use all OTUs

# Run UMAP analysis
transform_data=scaled_data.T
umap_model=umap.UMAP(n_components=1, metric='correlation', n_neighbors=UMAP_n_neighbors, min_dist=UMAP_min_distance, random_state=40).fit(transform_data.iloc[:,otu_select])

# Save UMAP coordinates
umap_result=pd.DataFrame(umap_model.embedding_)
umap_result=pd.concat([umap_result, metadata.reset_index(drop=True)],axis=1)
umap_result.to_csv('UMAP_sample.csv', sep=',')

## Visualize the temporal variation of each treatment
Use cubic smoothing splines to fit the UMAP coordinates of each treament

*   **curve_smooth:** 0 for very general fitting, 1 for overfitting



In [ ]:
curve_smooth=0.5 # between 0 and 1

# Get unique groups
groups = umap_result['Type']+'_'+umap_result['Site']
unique_groups = groups.unique()

# Interactive visualization
time_interval=0.1
fig = make_subplots()
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'white'})
color_palette = px.colors.qualitative.D3
for i, group in enumerate(unique_groups):
    group_data = umap_result[groups == group].reset_index(drop=True)
    group_data['Time']=group_data['Time']+np.random.rand(group_data.shape[0])/1000
    timevec=np.arange(group_data['Time'].min(), group_data['Time'].max()+time_interval, time_interval)
    spline_result = csaps(group_data['Time'][np.argsort(group_data['Time'])], group_data[0][np.argsort(group_data['Time'])], timevec, smooth=curve_smooth)

    color = color_palette[i % len(color_palette)]
    fig.add_trace(go.Scatter(x=group_data['Time'], y=group_data[0], mode='markers', name=group, marker=dict(color=color),
    showlegend=False ))
    fig.add_trace(go.Scatter(x=timevec, y=spline_result, mode='lines', name=group + ' (smooth)', line=dict(color=color)))
fig.update_yaxes(title_text='UMAP Coordinate', range=[np.floor(umap_result[0].min()), np.ceil(umap_result[0].max())], linewidth=2, linecolor='black', mirror=True)
fig.update_xaxes(title_text='Time', range=[np.floor(umap_result['Time'].min())-1, np.ceil(umap_result['Time'].max())+1], linewidth=2, linecolor='black', mirror=True)
fig.show()
fig.write_html(file='UMAP_sample_ts.html')

save_figure=True
if save_figure:
  fig.write_image("UMAP_sample_ts.jpg", height=400, width=1000, scale=5)

## Visualize UMAP result
Produce a heatmap of mean abundance (bin determined by *umap_bin_width*) to summarize sample patterns and their links to OTU abundances

In [ ]:
umap_bin_width=1 # value >0

# Create feature matrix
k=0
pick_umap=np.arange(np.floor(umap_result[0].min()),np.ceil(umap_result[0].max()),umap_bin_width)
W=np.zeros((len(pick_umap),transform_data.shape[1]))+np.nan
for n in pick_umap:
    ind=np.where((umap_result[0]>=n)*(umap_result[0]<(n+umap_bin_width))==1)[0]
    W[k,:]=np.mean(transform_data.iloc[ind,:],axis=0)
    k+=1

# Interactive visualization
fig = make_subplots(rows=1, cols=1)
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'white'}, height=600)
fig.add_trace(go.Heatmap(z=W[:,otu_vector], y=pick_umap+umap_bin_width/2, x=data['#OTU'].iloc[otu_vector], coloraxis='coloraxis1'), row=1, col=1)
fig.update_yaxes(title_text='UMAP Coordinate', range=[np.floor(umap_result[0].min()), np.ceil(umap_result[0].max())], showgrid=False, zeroline=False, showline=True, linewidth=2, linecolor='black', mirror=True, row=1, col=1)
fig.update_xaxes(title_text='Sample', showgrid=False, zeroline=False, showline=True, linewidth=2, linecolor='black', mirror=True, row=1, col=1)
fig.update_layout(coloraxis1=dict(colorscale='Jet', showscale=True), bargap=0.05)
fig.show()
fig.write_html(file='UMAP_Sample_1D.html')

save_figure=True
if save_figure:
  fig.write_image("UMAP_Sample_1D.jpg", height=400, width=1000, scale=5)